https://assaeunji.github.io/machine%20learning/2020-11-29-implicitfeedback/

# ALS 모델 기반 학습 및 평가 진행
- 메디스트림의 order 데이터 기반으로 아이템 추천을 진행합니다.
- 3개월치 데이터 중 도서 카테고리 아이템 추천을 진행합니다.
- train test 나눈 후 prediction 과 real item의 NDCG score를 통해 평가합니다.
- ALS 모델과 Most popular prediction 비교 합니다.

In [825]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS
from mlxtend.frequent_patterns import apriori

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/user_3/medistream-recsys/Script


# 1.Dataload

In [826]:
df = pd.read_json('/fastcampus-data/select_column_version_3.json')

In [827]:
date_state = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]
# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > date_state]
# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
# 도서 카테고리만 가져오기
only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)

# 도서, 소모품 카테고리
# df_book = complete_df[complete_df['name'].isin(['도서','소모품'])].sort_values(by='date_paid')

In [828]:
# none 값 확인하기
df_book.isna().sum()

_id                0
date_paid          0
customer_id        0
paid               0
name_x             0
category_id_y      0
product_ids        0
quantity           0
price              0
price_total        0
age_group        905
한의사 여부             1
사업자 여부             1
cancelled          0
name               0
slug               0
dtype: int64

## 전체 데이터 EDA

In [829]:
print('중복 제거 전:',len(only_book), '중복 제거 후:',len(df_book))

중복 제거 전: 5931 중복 제거 후: 5910


In [830]:
print('전체 데이터 수:',len(df_book))

전체 데이터 수: 5910


In [831]:
print('아이템 수:',len(df_book.product_ids.unique()),'유저 수:',len(df_book.customer_id.unique()))

아이템 수: 252 유저 수: 2663


# 2.train test split
- 마지막 3주 분량을 test로 선정합니다.
- train 3번 이상 구매 유저에 대해서만 학습을 진행합니다.
- test 3번이상 구매 유저를 대상으로 최근 3번 구매한 기록까지 예측을 진행합니다.
- train 있는 유저만을 test 평가를 진행합니다.

In [832]:
df_book['date_paid'].max()

Timestamp('2022-09-13 08:51:40+0000', tz='UTC')

In [833]:
from datetime import datetime, timedelta

date = df_book['date_paid'].max()-timedelta(weeks=3)
train_before_preprocess = df_book[df_book['date_paid'] < date]
test_before_preprocess = df_book[df_book['date_paid'] >= date]

# train 3개 이상 구매 기록유저에 대한 학습 진행
train_drop_row_index = train_before_preprocess['customer_id'].value_counts()[train_before_preprocess['customer_id'].value_counts()>=3].index
train = train_before_preprocess.copy()
train_mf = train_before_preprocess[train_before_preprocess['customer_id'].isin(train_drop_row_index)]
train_ap_mp = train_before_preprocess[~(train_before_preprocess['customer_id'].isin(train_drop_row_index))]

## 전체 아이템 중복 확인

In [834]:
# product_ids, name_x 수는 일치
len(df_book.product_ids.unique()), len(df_book.name_x.unique())

(252, 252)

In [835]:
# 중복 제거 후 수 비교 확인
# 252로 일치하여 문제 없음
len(df_book.drop_duplicates(subset=['product_ids','name_x']).name_x.unique())

252

## train test 아이템 중복 확인

In [836]:
len(train_before_preprocess.product_ids.unique()),len(test_before_preprocess.product_ids.unique())

(243, 130)

In [837]:
len(set(train_before_preprocess.product_ids.unique())-set(test_before_preprocess.product_ids.unique()))

122

In [838]:
# test 아이템에 train 없는 아이템 확인
len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique()))

9

In [839]:
# test 만 있는 item 제거
only_test_items = set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())
test = test_before_preprocess[~test_before_preprocess['product_ids'].isin(only_test_items)]

In [840]:
len(test.customer_id.unique()), len(train_before_preprocess.customer_id.unique())

(491, 2323)

In [841]:
len(test.customer_id.unique())

491

# train test eda

### 전처리 전후 비교

In [842]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 5024 train 전처리 후: 5024


In [843]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 886 test 전처리 후: 651


### user 수 비교 

In [844]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 2323


In [845]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 491


In [846]:
# 신규 유저는 MP 같은 다른 방법으로 추천 진행해야 함
print('test 만 있는 신규 유저 :',len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))

test 만 있는 신규 유저 : 240


### item 개수 비교

In [847]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 243 test 아이템 수 : 121


In [848]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 122


In [849]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 0


In [850]:
# test 만 있는 신규 유저 아이템 구매 횟수
test[test['customer_id'].isin(set(test['customer_id'].unique())- set(train['customer_id'].unique()))].customer_id.value_counts()

607ffd14c1908b001a63efbf    8
602a2d99527223001a2091d0    6
5e5f01c7bfe4260944782d10    5
6225cd53f967570023fa9c23    5
604ee9970ccc56001bc78b2a    4
5d913dbe0dabe405b156d5bd    3
60f5138d09c63a06ed1473f0    3
5e0444bc4267e105dfff00ba    3
60505ffa0ccc56001bc78d10    3
5d9aa6f50dabe405b156e103    3
5e04f3424267e105dfff00f5    3
5f2cfd57a0b91001bf48bb14    3
616f635d4cf9a50022880b67    2
5d8d61a10dabe405b156d566    2
5d92bfba0dabe405b156d5fb    2
5d8351333f0e6805c4706818    2
5e902e02bfe4260944785236    2
61922390bc63410023477425    2
5e01d8a24267e105dffeff76    2
5df05ce74267e105dffef3b0    2
5db965840dabe405b156f955    2
62943d3d9d93880024070553    2
5de855804267e105dffeec09    2
5d9a86b80dabe405b156decb    2
5d96ab5d0dabe405b156db10    2
5d9eec5f0dabe405b156e25a    2
5ec75f07e510ee503167b4e1    2
5df18b2b4267e105dffef4b1    2
5dd1f9782bb59605ca3d00f0    2
5e9d5ecdbfe4260944785ead    2
5e8fe576bfe4260944785169    2
5eb2566b1a3ca36b9678527d    2
5d96bec00dabe405b156db4e    2
613810003b

In [851]:
test[~test['customer_id'].isin(set(test['customer_id'].unique())- set(train['customer_id'].unique()))].customer_id.value_counts()

5ff7fdc7d6491c001b1785f6    13
602ac6c22c593a001acea9c9     7
60239968527223001a208810     5
5ebb5aa409982e0735b2d76b     5
5d9708790dabe405b156dccc     5
5dce69512bb59605ca3cffb6     4
5e53b0f1bfe42609447827e4     4
5e5da336bfe4260944782c29     4
5f560405a0b91001bf48f643     4
5fa0749351203163343212a3     4
5e1011394267e105dfff0538     4
6076dac90ccc56001bc7b4e1     3
5ec3586fe510ee503167aa4c     3
5df191784267e105dffef5d1     3
6220687af967570023fa7e74     3
5d6f735b19efa30eb2914094     3
5d61fb634e77525ec5ca1585     3
5d9690200dabe405b156d8c0     3
5e5fb67cbfe4260944782d88     3
5fad04025120316334322cb8     3
5e6c4d85bfe42609447836eb     3
5df197a54267e105dffef683     2
5f54931ea0b91001bf48f3f9     2
5d8ae1b90dabe405b156d4f6     2
5e622126bfe42609447830ca     2
60a6bb9e0ccc56001bc7e949     2
5e674c5fbfe426094478334d     2
5d9730bd0dabe405b156dd50     2
5ea8d01e1a3ca36b96784e0f     2
5db644a60dabe405b156f6bc     2
60178b4e2c593a001ace9279     2
5f1022eb221c8b5f77c8e5bb     2
5e81b93f

MF 추천시 MP와 MF 거의 동일한 비율로 각각 추천이 되므로    
각 추천의 영향력 또한 NDCG 매길 시 비슷하게 나올 것으로 판단됨

## train_mf 와 train_ap_mp 비교

In [852]:
print('train_mf 수:',len(train_mf), 'train_ap_mp 수:',len(train_ap_mp))

train_mf 수: 2745 train_ap_mp 수: 2279


In [853]:
print('train_mf 유저 수:',len(train_mf.customer_id.unique()) , 'train_ap_mp 유저 수:',len(train_ap_mp.customer_id.unique()))

train_mf 유저 수: 557 train_ap_mp 유저 수: 1766


In [854]:
print('train_mf 아이템 수:',len(train_mf.product_ids.unique()) , 'train_ap_mp 아이템 수:',len(train_ap_mp.product_ids.unique()))

train_mf 아이템 수: 231 train_ap_mp 아이템 수: 162


In [855]:
# 비신규 유저 비율이 train_mf 와 train_ap_mp 중 어디가 높은 지 확인

# test 비신규 유저 set
new_users = set(test['customer_id'].unique())-set(train['customer_id'].unique()) 
non_new_user_test = test[~test['customer_id'].isin(new_users)]
print('비신규 test 유저 수:', len(set(non_new_user_test.customer_id)))

비신규 test 유저 수: 251


In [856]:
print('train_mf 신규 유저 수:', len(set(non_new_user_test.customer_id)-set(train_mf.customer_id)))
print('train_ap_mp 신규 유저 수:', len(set(non_new_user_test.customer_id)-set(train_ap_mp.customer_id)))

train_mf 신규 유저 수: 137
train_ap_mp 신규 유저 수: 114


In [857]:
# 신규 유저 수
len(set(test['customer_id'].unique())-set(train['customer_id'].unique()))

240

- 결론, train_mf : 137 명, train_ap_mp : 354 명
- 두 배정도 높은 비율로 mp가 추천이 됨

# Apriori 분석

In [858]:
#Apriori treats duplicates as a single occurence
# 한 유저당 구매한 제품들의 unique -> 중복을 없앰.
train_apri = train.groupby('customer_id')['name_x'].unique().reset_index()
# train_apri[:30]

In [859]:
# list 담긴 중복 없는 아이템 
# date_paid 까지 포함하여 groupby 했으나 기간 고려 안함
from mlxtend.preprocessing import TransactionEncoder
transactions = [a[1]['name_x'].tolist() for a in list(train.groupby(['customer_id']))]

In [860]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

In [861]:
orders_1hot = pd.DataFrame(te_ary, columns=te.columns_)
pf = orders_1hot.describe()

In [862]:
f = pf.iloc[0]-pf.iloc[3] # count - frequent
a = f.tolist()
b = list(f.index)
item = pd.DataFrame([[a[r],b[r]]for r in range(len(a))], columns=['Count','Item'])
item = item.sort_values(['Count'], ascending=False).head(10)
print("MOST POPULAR")
item

MOST POPULAR


,Count,Item
84,574,비만문답
211,447,플로차트 정형외과 진단
242,313,흔히보는 정형외과 외래진료 가이드북
144,222,윤상훈·권병조의 알짜 근육학
241,213,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼
240,197,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록
131,143,외래에서 꼭 알아야 할 통증증후군 137가지
196,133,카이로프랙틱 기본테크닉론
90,125,사암침의 해석과 임상
126,101,약침의 정석 –통증편


In [863]:
from mlxtend.frequent_patterns import apriori
is_ap = apriori(orders_1hot, min_support=0.09, max_len=None, use_colnames=True)

In [864]:
from mlxtend.frequent_patterns import association_rules
rules_confidence = association_rules(is_ap, metric='confidence', min_threshold=0.03) # default가 confidence

display(rules_confidence)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


apriori 통한 최소한의 예측을 진행하기 위한 support 와   
confidence 가 너무 낮아서 진행하지 못한다고 판단함

# 3. sparse matrix 만들기

### ALS matrix

In [865]:
PdIds = train_mf.product_ids.unique()

PdIdToIndex = {}
indexToPdId = {}

colIdx = 0

for PdId in PdIds:
    PdIdToIndex[PdId] = colIdx
    indexToPdId[colIdx] = PdId
    colIdx += 1
    
userIds = train_mf.customer_id.unique()

userIdToIndex = {}
indexToUserId = {}

rowIdx = 0

for userId in userIds:
    userIdToIndex[userId] = rowIdx
    indexToUserId[rowIdx] = userId
    rowIdx += 1

import scipy.sparse as sp

rows = []
cols = []
vals = []

for row in train_mf.itertuples():
    rows.append(userIdToIndex[row.customer_id])
    cols.append(PdIdToIndex[row.product_ids])
    vals.append(1)

purchase_sparse = sp.csr_matrix((vals, (rows, cols)), shape=(rowIdx,colIdx))

matrix = purchase_sparse.todense()
matrix

matrix([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

### Most_popular_matrix

In [870]:
most_popular = train.groupby(['product_ids','name_x']).count()['customer_id'].reset_index()
most_popular

,product_ids,name_x,customer_id
0,5d13115e32026c0b35383897,KCD 한방내과 진찰진단 가이드라인,37
1,5d5373e94e77525ec5ca1135,통증치료를 위한 근육 초음파와 주사 테크닉,18
2,5d59ee854e77525ec5ca1212,일차진료 한의사를 위한 보험한약입문 - 둘째 판,40
3,5d77b31f19efa30eb29143c9,NEO 인턴 핸드북,19
4,5d77b73619efa30eb29143ca,병태생리 Visual map,8
5,5d78491b19efa30eb29143cc,플로차트 한약치료,48
6,5d784bac19efa30eb29143d0,Medical acupuncture '침의 과학적 접근과 임상활용',9
7,5d784e1719efa30eb29143d1,"근골격계 질환의 진단 및 재활치료, 4판",9
8,5d78505019efa30eb29143d2,개원의를 위한 통증사냥법,29
9,5d7852c519efa30eb29143d3,Cyriax 정형의학 3판,8


# Sparsity 확인

In [871]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

97.86658583786053

# 4. Model

# Model 학습 진행
- real test 만들기
- implict 라이브러리 사용(MF,LMF)
- MF 구현 모델 사용

In [872]:
# real test 
ground_trues = []
for user_id in test['customer_id'].unique():
    ground_trues.append({'id': user_id,\
    'items':list(test[test['customer_id']==user_id].product_ids)
    })

## ALS fit

In [873]:
als_model = ALS(factors=20, regularization=0.01, iterations = 50)
als_model.fit(purchase_sparse)

  0%|          | 0/50 [00:00<?, ?it/s]

# 5. prediction

## MP & als MF prediction

In [880]:
# train 구매 3미만인 경우 및 신규 유저인 경우 mp로 넣기
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
als_mp_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        als_mp_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        als_mp_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(als_mp_predict_list):
    als_mp_predict_list[idx]['items'] = pred_list['items'][:100]

# most popular prediction

In [881]:
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
predict_popular_list = []
for user_id in test['customer_id'].unique():
    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
    predict_popular_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                        if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                        ]})

# 100 개만 예측하기
for idx, pred_list in enumerate(predict_popular_list):
    predict_popular_list[idx]['items'] = pred_list['items'][:100]

# 6. evaluation

## NDCG 평가지표

In [882]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)


        return ndcg_score

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score = self._eval(gt_list, rec_list)
            print(f"nDCG: {ndcg_score:.6}")
        except Exception as e:
            print(e)


# MP & als MF prediction NDCG

In [883]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, als_mp_predict_list)

nDCG: 0.248479


In [884]:
len(als_mp_predict_list),len(ground_trues)

(491, 491)

In [885]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, als_mp_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

495

# most popular NDCG

In [886]:
# most popular
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, predict_popular_list)

nDCG: 0.262317


In [887]:
len(predict_popular_list),len(ground_trues)

(491, 491)

In [888]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, predict_popular_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

546

In [889]:
for item in predict_popular_list[4]['items'][:5]:
    print(df_book[df_book['product_ids']==item].name_x.unique())

['윤상훈·권병조의 알짜 근육학']
['외래에서 꼭 알아야 할 통증증후군 137가지']
['카이로프랙틱 기본테크닉론']
['약침의 정석 –통증편']
['임상 한의사를 위한 기본 한약처방 강의 2판']


# 7. 추천된 items 확인

In [890]:
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 688691.05it/s]


In [891]:
# pred_item, rea_item 비교
def pred_real_dataframe(user_num):
    pred_items_names = []
    predict_dict = als_mp_predict_list[user_num]['items']
    for item in predict_dict:
        pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())

    real_items_names = []
    trues_dict = ground_trues[user_num]['items']
    for item in trues_dict:
        real_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())
    return pd.DataFrame({'pred_item':pred_items_names,'real_item':real_items_names})
    

In [892]:
# train_item, pred_item, real_item 비교
def train_pred_items(user_nums):
    train_pred_items_df = pd.DataFrame(columns=['train_item','pred_item'])
    for user_num in range(1,user_nums):
        train_item_names = []
        for idx in grouped_purchased[grouped_purchased['customer_id']==ground_trues[user_num]['id']].product_ids:
            train_item_names.append(products_df[products_df['_id'] == idx].meta_title.unique()[0])

        pred_items_names = []
        predict_dict = als_mp_predict_list[user_num]['items']
        for item in predict_dict:
            pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())


        
        train_pred_items_df.loc[user_num,'train_item'] = train_item_names
        train_pred_items_df.loc[user_num,'pred_item'] = pred_items_names
    return train_pred_items_df


In [893]:
train_pred_items(13)

,train_item,pred_item
1,"[그림과 사진으로 보는 질환별 통증치료 Essential, 대학경락경혈학총론 & 대...","[[藥徵, 약의 징표], [트리거포인트 침치료], [사상임상약물대전], [최강통증매..."
2,[],"[[비만문답], [플로차트 정형외과 진단], [흔히보는 정형외과 외래진료 가이드북]..."
3,[],"[[비만문답], [플로차트 정형외과 진단], [흔히보는 정형외과 외래진료 가이드북]..."
4,"[흔히보는 정형외과 외래진료 가이드북, 趙紹琴(조소금) 내과학, 침구과 진료매뉴얼,...","[[SMART 기본 일차진료매뉴얼 3판(세트)], [초음파 유도하 침 시술 가이드북..."
5,[],"[[비만문답], [플로차트 정형외과 진단], [흔히보는 정형외과 외래진료 가이드북]..."
6,[],"[[비만문답], [플로차트 정형외과 진단], [흔히보는 정형외과 외래진료 가이드북]..."
7,"[흔히보는 정형외과 외래진료 가이드북, 초음파 가이드 근골격계 통증 치료의 정석, ...","[[감별진단의 정석], [숨찬 세상, 호흡기를 편하게], [트리거포인트 침치료], ..."
8,[],"[[비만문답], [플로차트 정형외과 진단], [흔히보는 정형외과 외래진료 가이드북]..."
9,[],"[[비만문답], [플로차트 정형외과 진단], [흔히보는 정형외과 외래진료 가이드북]..."
10,[],"[[비만문답], [플로차트 정형외과 진단], [흔히보는 정형외과 외래진료 가이드북]..."


In [894]:
# 예측 유저 구매 횟수 확인
pd.DataFrame(purchase_sparse[1].todense()).T.value_counts()

0    207
1     24
dtype: int64